In [1]:
## Advanced Course in Machine Learning
## Week 7
## Exercise 2 / Neural networks for sequences

import numpy as np
import scipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import linalg as LA
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Embedding
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.layers import LSTM

import keras


Using TensorFlow backend.


In [2]:
# Read in the training data
# Path /home/mkotola/AdvancedML/Week7
dataFile = "seqtrain.csv"
data = pd.read_csv(dataFile, sep=",", header=None)
# N = 4000, d = 31

In [3]:
# Read in the validation data
validDataFile = "seqvalid.csv"
validData = pd.read_csv(validDataFile, sep=",", header=None)

In [4]:
y = data.iloc[:,29:33]
X = data.iloc[:,0:29]
y_valid = validData.iloc[:,29:33]
X_valid = validData.iloc[:,0:29]

In [5]:
#print(X.mean(axis=0))
#print(X.std(axis=0))

In [6]:
# Scale the test data
sc = StandardScaler()
X = sc.fit_transform(X)
X_valid = sc.transform (X_valid)

In [7]:
#print(X.mean(axis=0))
#print(X.std(axis=0))

In [8]:
#print(y)

In [9]:
def codeYClassr(orig):  
    y = orig.copy()
    y['yclass'] = np.where(y.iloc[:,0]==1, 29, 0)
    y['yclass'] = np.where(y.iloc[:,1]==1, 30, y.iloc[:,4])
    y['yclass'] = np.where(y.iloc[:,2]==1, 31, y.iloc[:,4])
    y['yclass'] = np.where(y.iloc[:,3]==1, 32, y.iloc[:,4])
    return y

In [10]:
y_cl = codeYClassr(y)
y_val_cl = codeYClassr(y_valid)
y_train = y_cl['yclass']
y_validate = y_val_cl['yclass']

In [12]:
# Logistic regression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X, y_train)

In [13]:
pred = clf.predict(X_valid)

In [15]:
result = y_validate.eq(pred)
print(np.sum(result)/result.size)

0.24


In [16]:
# Logistic regression is able to classify the validation instances only 24 % correctly - which is worse than random
# guessing! Random guessing would result in 25 % correct labeling with 4 classes.

In [18]:
# Keras model / MLP
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=29))
model.add(Dense(units=64, activation='relu', input_dim=64))
model.add(Dense(units=4, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [71]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1920      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 6,340
Trainable params: 6,340
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

#model.compile(loss=keras.losses.categorical_crossentropy, 
#optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

In [20]:
model.fit(X, y, epochs=5, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
4000/4000 [==============================] - 0s 63us/step - loss: 1.3832 - acc: 0.2978
Epoch 2/5
4000/4000 [==============================] - 0s 29us/step - loss: 1.3240 - acc: 0.3465
Epoch 3/5
4000/4000 [==============================] - 0s 29us/step - loss: 1.2744 - acc: 0.3995
Epoch 4/5
4000/4000 [==============================] - 0s 29us/step - loss: 1.2256 - acc: 0.4357
Epoch 5/5
4000/4000 [==============================] - 0s 28us/step - loss: 1.1776 - acc: 0.4585


In [21]:
loss_and_metrics = model.evaluate(X_valid, y_valid, batch_size=128)

1000/1000 [==============================] - 0s 34us/step


In [22]:
print(loss_and_metrics)

[1.1783238515853882, 0.4220000014305115]


In [23]:
# Keras model / CNN A
model_m = Sequential()
model_m.add(Conv1D(100, 3, activation='relu', input_shape=(29, 1)))
model_m.add(Conv1D(100, 3, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(160, 3, activation='relu'))
model_m.add(Conv1D(160, 3, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(0.5))
model_m.add(Dense(4, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [24]:
print(model_m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 27, 100)           400       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 25, 100)           30100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 100)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 6, 160)            48160     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 4, 160)            76960     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 160)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 160)               0         
__________

In [25]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_m.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [26]:
X_exp = np.expand_dims(X, axis=2)

In [27]:
X_valid_exp = np.expand_dims(X_valid, axis=2)

In [28]:
#model.fit(X, y, epochs=5, batch_size=32)
#model.fit(X, y,
#              batch_size=32,
#              epochs=5,
#              validation_data=(X_valid, y_valid), shuffle=True)
model_m.fit(X_exp, y, batch_size=32, epochs=20)

Epoch 1/20
4000/4000 [==============================] - 1s 341us/step - loss: 1.3826 - acc: 0.2787
Epoch 2/20
4000/4000 [==============================] - 1s 265us/step - loss: 1.3591 - acc: 0.3325
Epoch 3/20
4000/4000 [==============================] - 1s 278us/step - loss: 1.3077 - acc: 0.3847
Epoch 4/20
4000/4000 [==============================] - 1s 277us/step - loss: 1.2518 - acc: 0.3965
Epoch 5/20
4000/4000 [==============================] - 1s 274us/step - loss: 1.2063 - acc: 0.4185
Epoch 6/20
4000/4000 [==============================] - 1s 272us/step - loss: 1.1764 - acc: 0.4360
Epoch 7/20
4000/4000 [==============================] - 1s 271us/step - loss: 1.1621 - acc: 0.4417
Epoch 8/20
4000/4000 [==============================] - 1s 272us/step - loss: 1.1404 - acc: 0.4517
Epoch 9/20
4000/4000 [==============================] - 1s 271us/step - loss: 1.1279 - acc: 0.4607
Epoch 10/20
4000/4000 [==============================] - 1s 273us/step - loss: 1.1199 - acc: 0.4605
Epoch 11/

In [29]:
loss_and_metrics = model_m.evaluate(X_valid_exp, y_valid, batch_size=128)
print(loss_and_metrics)

1000/1000 [==============================] - 0s 137us/step
[1.0624894580841064, 0.5019999985694885]


In [31]:
# Keras LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(1024, output_dim=256))
model_lstm.add(LSTM(128))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(4, activation='softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model_m.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [66]:
# Keras LSTM
model_l = Sequential()

#model_l.add(LSTM(32, input_shape=(29,1)))  # returns a sequence of vectors of dimension 32

model_l.add(LSTM(32, return_sequences=True,
               input_shape=(29, 1)))  # returns a sequence of vectors of dimension 32
model_l.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model_l.add(LSTM(32))  # return a single vector of dimension 32
model_l.add(Dense(4, activation='softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model_l.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [67]:
model_l.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 29, 32)            4352      
_________________________________________________________________
lstm_20 (LSTM)               (None, 29, 32)            8320      
_________________________________________________________________
lstm_21 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 132       
Total params: 21,124
Trainable params: 21,124
Non-trainable params: 0
_________________________________________________________________


In [68]:
model_l.fit(X_exp, y, batch_size=16, epochs=10)

Epoch 1/10
4000/4000 [==============================] - 9s 2ms/step - loss: 1.3860 - acc: 0.2567
Epoch 2/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.3841 - acc: 0.2705
Epoch 3/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.3710 - acc: 0.2765
Epoch 4/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.3153 - acc: 0.3588
Epoch 5/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.2431 - acc: 0.4178
Epoch 6/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.1929 - acc: 0.4465
Epoch 7/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.1641 - acc: 0.4577
Epoch 8/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.1384 - acc: 0.4675
Epoch 9/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.1154 - acc: 0.4830
Epoch 10/10
4000/4000 [==============================] - 7s 2ms/step - loss: 1.0952 - acc: 0.4963


In [70]:
loss_and_metrics = model_l.evaluate(X_valid_exp, y_valid, batch_size=16)
print(loss_and_metrics)

1000/1000 [==============================] - 1s 807us/step
[1.0967765607833861, 0.479]
